In [1]:
import pandas as pd
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Production Version 

In [17]:
df2 = pd.read_csv('CHI_sites_azimuths_0611_2_with_traffic_flag_height in meters.csv')

In [227]:

# Function to calculate the angle between two points
def calculate_angle(x1, y1, x2, y2):
    angle = np.arctan2(y2 - y1, x2 - x1)
    angle = np.degrees(angle)
    return angle if angle >= 0 else angle + 360



def NBR_(lat,lon,az,ang,site_id,cell_id, df):
    x_lat, x_lon = lat,lon
 #   print(az)
    # Calculate the angle for each site
    df['angle'] = df.apply(lambda row: calculate_angle(x_lat, x_lon, row['Latitude'], row['Longitude']), axis=1)
 ####   print(df[df['TX ID']=='CHCHI00920A_n71_G_1']['angle'])
    # Filter the sites based on the desired angle range
    desired_angle = az
    angle_range = ang
   # print(df[df['Site ID']=='CHCHI00377A'])
    criup = desired_angle - angle_range
   # print(criup)
    criup = [ criup if criup > 0 else 360+criup][0]
   # print(criup)    
    cridwn = desired_angle + angle_range
  #  print(cridwn)
    cridwn = [ cridwn if cridwn < 360 else cridwn-360][0]
   # print(cridwn)
 #   print( criup, cridwn)
 #   print(df['angle'].max(),df['angle'].min())
 #   print(criup, cridwn)

    filtered_df_1 = df[(df['angle'] >= criup) & (df['angle'] <= desired_angle)]
    filtered_df_2 = df[(df['angle'] >= desired_angle) & (df['angle'] <= cridwn)]
    filtered_df = pd.concat([filtered_df_1,filtered_df_2])
 #   filtered_df = pd.merge(df[df['angle'] >= criup],df[df['angle'] <= cridwn], on='Site ID', how='inner')


    mapping ={'Sector ID_x':'Sector ID' ,
        'Antenna X CL Rad Height (ft)_x':'Antenna X CL Rad Height (ft)',
        'Antenna X Antenna Model #_x': 'Antenna X Antenna Model #',
        'Antenna X Mech Downtilt_x':'Antenna X Mech Downtilt',
        'Antenna X Elec Downtilt_x':'Antenna X Elec Downtilt',
        'Antenna X Azimuth_x':'Antenna X Azimuth',
        'TX ID_x':'TX ID',
        'NR Cell Name_x':'NR Cell Name',
        'PCI_x':'PCI',
        'Latitude_x':'Latitude', 
        'Longitude_x':'Longitude',
        'Traffic_flag_x':'Traffic_flag', 
        'Unnamed: 13_x':'Unnamed: 13', 
        'angle_x':'angle'}
    filtered_df=filtered_df.rename(columns=mapping)


 #   print(filtered_df.columns)
  #  print(filtered_df['Site ID'].count())
#    filtered_df = df[(df['angle'] >= criup) & (df['angle'] <= desired_angle + angle_range)]
    target_location = (lat, lon)
    #print(filtered_df)
    # Calculate distance
  #  print(filtered_df.count())
#    print(filtered_df.apply(lambda row: geodesic(target_location,(row['Latitude'],  row['Longitude'])).meters, axis=1))
 #   print("SiteID: CHCHI00803A",df[df['Site ID'] == 'CHCHI00803A']['angle'])
#    print("SiteID: CHCHI00713B",df[df['Site ID'] == 'CHCHI00713B']['angle'])
 #  print("SiteID: CHCHI00083A",df[df['Site ID'] == 'CHCHI00083A']['angle'])    
    filtered_df['distance'] = filtered_df.apply(lambda row: geodesic(target_location,(row['Latitude'],  row['Longitude'])).meters, axis=1)

    # Filter out cells more than 90 degrees away from the target azimuth
    df_filtered2 = filtered_df[filtered_df['Site ID'] != site_id]
    result = []
    # Find the nearest cell with the closest azimuth
  #  temp = df_filtered2.sort_values('distance')[['TX ID','distance','angle','Latitude','Longitude','Antenna X Azimuth']].head(10)
   # temp['angle2'] = temp.apply(lambda row: abs(calculate_angle(row['Latitude'], row['Longitude'], lat,lon)), axis=1)
  #  temp['criteria'] = temp.apply(lambda row: row['Antenna X Azimuth'] - row['angle2'] 
   #                                        if row['Antenna X Azimuth'] - row['angle2'] >= 0 
   #                                        else row['Antenna X Azimuth'] - row['angle2'] + 360,
   #                                        axis=1
   #                                        )
  #  print(temp[['TX ID','Antenna X Azimuth','distance','criteria']])
    firstnearest_cell = df_filtered2.loc[df_filtered2[['distance']].idxmin().sum()]
    df_filtered3 = df_filtered2[df_filtered2['TX ID'] != firstnearest_cell['TX ID']]
    sec_nearest_cell = df_filtered3.loc[df_filtered3[['distance']].idxmin().sum()]
    
    df_filtered4 = df_filtered3[df_filtered3['TX ID'] != sec_nearest_cell['TX ID']]
    third_nearest_cell = df_filtered4.loc[df_filtered4[['distance']].idxmin().sum()]

    result.append({"first": firstnearest_cell['TX ID'] ,"second":sec_nearest_cell['TX ID'],
    "third":third_nearest_cell['TX ID']})
    
    out_df = df[df['TX ID'].isin([firstnearest_cell['TX ID'],sec_nearest_cell['TX ID'],third_nearest_cell['TX ID']])][['TX ID','Antenna X Azimuth','Longitude','Latitude','Site ID']]
    out_df['angle2'] = out_df.apply(lambda row: abs(calculate_angle(row['Latitude'], row['Longitude'], lat,lon)), axis=1)
   # out_df['criteria'] = out_df['Antenna X Azimuth'] - out_df['angle2']
    
    out_df['criteria'] = out_df.apply(lambda row: row['Antenna X Azimuth'] - row['angle2'] 
                                           if row['Antenna X Azimuth'] - row['angle2'] >= 0 
                                           else row['Antenna X Azimuth'] - row['angle2'] + 360,
                                           axis=1
                                           )
 ####   print(firstnearest_cell['TX ID'],sec_nearest_cell['TX ID'],third_nearest_cell['TX ID'])
 #   print(out_df)
    _df = filtered_df.copy()
    _df['distance']=_df.apply(lambda row: geodesic(target_location,(row['Latitude'],  row['Longitude'])).meters, axis=1)
    _df = _df[_df['Site ID'] != site_id]

    _df['angle2'] = _df.apply(lambda row: abs(calculate_angle(row['Latitude'], row['Longitude'], lat,lon)), axis=1)
   # out_df['criteria'] = out_df['Antenna X Azimuth'] - out_df['angle2']
    #
    _df['criteria'] = _df.apply(lambda row: abs(row['Antenna X Azimuth'] - row['angle2'] ),
#                                           if row['Antenna X Azimuth'] - row['angle2'] >= 0 
 #                                          else row['Antenna X Azimuth'] - row['angle2'] + 360,
                                           axis=1
                                           )
    
    nearest_site = _df.loc[_df[['distance']].idxmin().sum()]['Site ID']
  #  print("first",nearest_site)
    _df_ = _df[_df['Site ID']==nearest_site]
    top_1 = _df_.nsmallest(1, 'criteria')['TX ID'].iloc[0]
  #  site = _df_[_df_['TX ID']==top_1]['Site ID'].iloc[0]
    out_df2 = _df[_df['Site ID']!=nearest_site]
  #  print(out_df2.count())


    nearest_site = out_df2.loc[out_df2[['distance']].idxmin().sum()]['Site ID']
  #  print("second",nearest_site)
    _df_ = out_df2[out_df2['Site ID']==nearest_site]
    top_2 = _df_.nsmallest(1, 'criteria')['TX ID'].iloc[0]
  #  site = _df_[_df_['TX ID']==top_1]['Site ID'].iloc[0]
    out_df3 = out_df2[out_df2['Site ID']!=nearest_site]
  #  print(out_df2.count())

    nearest_site = out_df3.loc[out_df3[['distance']].idxmin().sum()]['Site ID']
 #   print("third",nearest_site)
    _df_ = out_df2[out_df2['Site ID']==nearest_site]
    top_3 = _df_.nsmallest(1, 'criteria')['TX ID'].iloc[0]
  #  site = _df_[_df_['TX ID']==top_1]['Site ID'].iloc[0]
   # out_df2 = _df[_df['Site ID']!=nearest_site]
  #  print(out_df2.count())


# #   out_df2 = out_df2.loc[out_df2[['distance']].idxmin().sum()]
 #   top_2 = out_df2.nsmallest(1, 'criteria')['TX ID'].iloc[0]
 #   site = out_df2[out_df2['TX ID']==top_2]['Site ID'].iloc[0]
 #   out_df3 = out_df2[out_df2['Site ID']!=site]
#
 #   out_df3 = out_df3.loc[out_df3[['distance']].idxmin().sum()]
 #   top_3 = out_df3.nsmallest(1, 'criteria')['TX ID'].iloc[0]
 #   site = out_df3[out_df3['TX ID']==top_3]['Site ID'].iloc[0]
 #   out_df4 = out_df3[out_df3['Site ID']!=site]
  #  print(top_1,top_2,top_3)

    top_3_entries = out_df.nsmallest(3, 'criteria')
    top_3_tx_ids = out_df.nsmallest(3, 'criteria')['TX ID'].tolist()
    #print(top_3_tx_ids)

    
    return {"Source":cell_id,"1NBR":top_1,"2NBR":top_2,"3NBR":top_3}# out_df.loc[out_df['criteria'].idxmin()]['TX ID']

In [228]:
cell = 'CHCHI00060A_n71_G_2'
#cell = 'CHCHI00207A_n71_G_1'
celldf = df2[df2['TX ID']==cell]
lat = celldf['Latitude'].iloc[0]
lon = celldf['Longitude'].iloc[0]
azi = celldf['Antenna X Azimuth'].iloc[0]
site = celldf['Site ID'].iloc[0]
cell = celldf['TX ID'].iloc[0]
#print(site,azi,cell)
#print(df2.count())
b = NBR_(lat,lon, azi,40,site,cell,df2)
b

{'Source': 'CHCHI00060A_n71_G_2',
 '1NBR': 'CHCHI00664C_n71_G_3',
 '2NBR': 'CHCHI00079A_n71_G_3',
 '3NBR': 'CHCHI00064A_n71_G_3'}

In [ ]:
import tqdm
import warnings
warnings.filterwarnings('ignore')
output_ = []
for cell in tqdm.tqdm(df2.iterrows()):
    try:
        NBR = NBR_(cell[1]['Latitude']	,cell[1]['Longitude']	, cell[1]['Antenna X Azimuth']	, 40,
                   cell[1]['Site ID']	,cell[1]['TX ID']	,df2)
        output_.append(NBR)
        
    except:
       # print(e)
        try:
            NBR = NBR_(cell[1]['Latitude']	,cell[1]['Longitude']	, cell[1]['Antenna X Azimuth'],100,
                       cell[1]['Site ID']	,cell[1]['TX ID']	,df2)
            output_.append(NBR)
        except Exception as e:
            output_.append({"source": cell[1]['TX ID'], "Error":e})
            pd.DataFrame(output_).to_csv('TEMP6NBR_Production.csv')
            continue
pd.DataFrame(output_).to_csv('6NBR_Production.csv')

479it [01:33,  7.03it/s]

In [36]:
pd.DataFrame(output_).to_csv('________NBR_Production.csv')